In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(9, 18, 3)
        self.conv2 = nn.Conv1d(18, 36, 5)
        self.pool1 = nn.Conv1d(36, 18, 9, 4)
        self.fc1 = nn.LazyLinear(128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)
        x = torch.flatten(x, 1)     # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x

net = Net()

In [ ]:
data_file = '../data/sheik_v_fox_15_seconds_polar.pkl'
df = pd.read_pickle(data_file)

df

In [ ]:
X = df['training_inputs_polar']
X = np.stack(X, axis = 0)
X = X.reshape((48982, 9, 900))

y = df['is_sheik']
y = np.array(y)

print(X.shape)
print(y.shape)

In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.Tensor(X_train)
y_train_tensor = torch.Tensor(y_train)
X_val_tensor = torch.Tensor(X_val)
y_val_tensor = torch.Tensor(y_val)

# Create DataLoader for training and validation sets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
print(type(y[0]))
print(y[0])

In [ ]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    net.train()  # Set the model to training mode
    for inputs, labels in train_loader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = net(inputs)
        outputs = outputs.squeeze()

        # Compute the loss
        # print(type(outputs[0]))
        # print(type(labels[0]))
        # labels = labels.long()
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Validation
    net.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            outputs = net(inputs)
            outputs = outputs.squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Calculate accuracy
            predicted = torch.max(outputs)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        average_val_loss = val_loss / len(val_loader)
        accuracy = correct / total

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {average_val_loss:.4f}, Val Accuracy: {accuracy:.2%}')
